In [11]:
# optimizer with scaling and lin reg

import pandas as pd
import warnings
import gurobipy as gp
import matplotlib.pyplot as plt
from sklearn import tree
import seaborn as sns
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.compose import make_column_transformer
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
from sklearn.neural_network import MLPRegressor

df = pd.read_csv('csv_dateien/starcraftFinalcsvs/stackedRun.csv')

df = df[:1350]

dfTerranReg = df[df["fraction"] == "Terraner"]

X = dfTerranReg[["total_workers"]]
y = dfTerranReg["resource_mining"]

print(dfTerranReg)

# Split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=1
)

feat_transform = make_column_transformer(
    (StandardScaler(), ["total_workers", "total_army", "time"]),
)

scaler = StandardScaler()
lin_reg = make_pipeline(scaler, LinearRegression())
lin_reg.fit(X_train, y_train)

y_pred = lin_reg.predict(X_test)
print(f"The R^2 value in the test set is {r2_score(y_test, y_pred)}")

data = pd.read_csv('csv_dateien/starcraftFinalcsvs/stackedRun.csv')
data = data[1350:]

dfTerran = data[data["fraction"] == "Terraner"]


# ,player,total_workers,total_army_value,total_army,fraction,winner,replay_filename
dfTerran = dfTerran.drop(columns=[
                               "total_workers", "player", "replay_filename", "winner", "fraction", "Unnamed: 0", "Unnamed: 0.1", "total_army_value", "resource_consumption", "time", "total_army"])



# create variables
# create Model

m = gp.Model("Worker optimizer")
gppd.set_interactive()

w = gppd.add_vars(m, pandas_obj=dfTerran,
                  name="total_workers", vtype=gp.GRB.INTEGER)
r_ges = gppd.add_vars(m, pandas_obj=dfTerran,
                      name="resource_mining")
a = gppd.add_vars(m, pandas_obj=dfTerran,
                  name="total_army", vtype=gp.GRB.INTEGER)

m.update()
dfTerran = dfTerran.drop(columns=["resource_mining"])

feats = dfTerran
feats = pd.concat([w, feats], axis=1)
print(feats.head)

m.setObjective(a.sum(), gp.GRB.MAXIMIZE) # Zielfunktion
gppd.add_constrs(m, ((r_ges - 50*w)/100), gp.GRB.GREATER_EQUAL, a) # Gleichung 2
gppd.add_constrs(m, w + a, gp.GRB.GREATER_EQUAL, 0) # Gleichung 3
gppd.add_constrs(m, w + a, gp.GRB.LESS_EQUAL, 200) # Gleichung 4
m.update()

pred_constr = add_predictor_constr(m, lin_reg, feats, r_ges) # Einfügen des Regressiosmodells
pred_constr.print_stats()
m.optimize()
outputs = pred_constr.input_values
print(pred_constr.input_values)
"""s#



feats = feats.drop(columns=["time"])
print(feats)

# pred_constr = add_predictor_constr(m, lin_reg, feats, av)
# m.Params.NonConvex = 2


print(w.gppd.X)

print(
    "Maximum error in approximating the regression {:.6}".format(
        np.max(pred_constr.get_error())
    )
)







dfTerranOpt = dfTerranOpt.loc[:, features]

m.update()

dfTerranOpt[:10]





"""


      Unnamed: 0.1  Unnamed: 0       player  total_workers  total_army_value  \
1                1           1         Clem             46              5825   
6                6           6      Optimus             66              8000   
15              15          15   Drunkenboi             54              2950   
19              19          19   INnoVation             35              1750   
24              24          24   Drunkenboi             52              6625   
...            ...         ...          ...            ...               ...   
1338          1338        1338  CMStormPolt             51              2850   
1342          1342        1342     SpaceJam             52              1975   
1346          1346        1346     Kelazhur             28              1700   
1347          1347        1347        Light             46              1775   
1349          1349        1349         TYTY             61              8375   

      total_army  time  fraction  winne

's#\n\n\n\nfeats = feats.drop(columns=["time"])\nprint(feats)\n\n# pred_constr = add_predictor_constr(m, lin_reg, feats, av)\n# m.Params.NonConvex = 2\n\n\nprint(w.gppd.X)\n\nprint(\n    "Maximum error in approximating the regression {:.6}".format(\n        np.max(pred_constr.get_error())\n    )\n)\n\n\n\n\n\n\n\ndfTerranOpt = dfTerranOpt.loc[:, features]\n\nm.update()\n\ndfTerranOpt[:10]\n\n\n\n\n\n'

In [14]:
# optimizer with scaling and mlp regressor

import pandas as pd
import warnings
import gurobipy as gp
import matplotlib.pyplot as plt
from sklearn import tree
import seaborn as sns
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.compose import make_column_transformer
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
from sklearn.neural_network import MLPRegressor

df = pd.read_csv('csv_dateien/starcraftFinalcsvs/stackedRun.csv')

df = df[:1350]

dfTerranReg = df[df["fraction"] == "Terraner"]

X = dfTerranReg[["total_workers"]]
y = dfTerranReg["resource_mining"]

print(dfTerranReg)

# Split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=1
)

feat_transform = make_column_transformer(
    (StandardScaler(), ["total_workers", "total_army", "time"]),
)

scaler = StandardScaler()
lin_reg = make_pipeline(scaler, MLPRegressor(random_state=2, max_iter=10000))
lin_reg.fit(X_train, y_train)

y_pred = lin_reg.predict(X_test)
print(f"The R^2 value in the test set is {r2_score(y_test, y_pred)}")

data = pd.read_csv('csv_dateien/starcraftFinalcsvs/stackedRun.csv')
data = data[1350:]

dfTerran = data[data["fraction"] == "Terraner"]


# ,player,total_workers,total_army_value,total_army,fraction,winner,replay_filename
dfTerran = dfTerran.drop(columns=[
                               "total_workers", "player", "replay_filename", "winner", "fraction", "Unnamed: 0", "Unnamed: 0.1", "total_army_value", "resource_consumption", "time", "total_army"])



# create variables
# create Model

m = gp.Model("Worker optimizer")
gppd.set_interactive()

w = gppd.add_vars(m, pandas_obj=dfTerran,
                  name="total_workers", vtype=gp.GRB.INTEGER)
r_ges = gppd.add_vars(m, pandas_obj=dfTerran,
                      name="resource_mining")
a = gppd.add_vars(m, pandas_obj=dfTerran,
                  name="total_army", vtype=gp.GRB.INTEGER)

m.update()
dfTerran = dfTerran.drop(columns=["resource_mining"])

feats = dfTerran
feats = pd.concat([w, feats], axis=1)
print(feats.head)

m.setObjective(a.sum(), gp.GRB.MAXIMIZE) # Zielfunktion
gppd.add_constrs(m, ((r_ges - 50*w)/100), gp.GRB.GREATER_EQUAL, a) # Gleichung 2
gppd.add_constrs(m, w + a, gp.GRB.GREATER_EQUAL, 0) # Gleichung 3
gppd.add_constrs(m, w + a, gp.GRB.LESS_EQUAL, 200) # Gleichung 4
m.update()

pred_constr = add_predictor_constr(m, lin_reg, feats, r_ges) # Einfügen des Regressiosmodells
pred_constr.print_stats()
m.optimize()
outputs = pred_constr.input_values
print(pred_constr.input_values)
"""s#



feats = feats.drop(columns=["time"])
print(feats)

# pred_constr = add_predictor_constr(m, lin_reg, feats, av)
# m.Params.NonConvex = 2


print(w.gppd.X)

print(
    "Maximum error in approximating the regression {:.6}".format(
        np.max(pred_constr.get_error())
    )
)







dfTerranOpt = dfTerranOpt.loc[:, features]

m.update()

dfTerranOpt[:10]





"""


      Unnamed: 0.1  Unnamed: 0       player  total_workers  total_army_value  \
1                1           1         Clem             46              5825   
6                6           6      Optimus             66              8000   
15              15          15   Drunkenboi             54              2950   
19              19          19   INnoVation             35              1750   
24              24          24   Drunkenboi             52              6625   
...            ...         ...          ...            ...               ...   
1338          1338        1338  CMStormPolt             51              2850   
1342          1342        1342     SpaceJam             52              1975   
1346          1346        1346     Kelazhur             28              1700   
1347          1347        1347        Light             46              1775   
1349          1349        1349         TYTY             61              8375   

      total_army  time  fraction  winne

c:\Users\nicol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  warnings.warn(


The R^2 value in the test set is 0.8496094124896243
<bound method NDFrame.head of                          total_workers
1356  <gurobi.Var total_workers[1356]>
1358  <gurobi.Var total_workers[1358]>
1364  <gurobi.Var total_workers[1364]>
1366  <gurobi.Var total_workers[1366]>
1368  <gurobi.Var total_workers[1368]>
...                                ...
2648  <gurobi.Var total_workers[2648]>
2649  <gurobi.Var total_workers[2649]>
2650  <gurobi.Var total_workers[2650]>
2651  <gurobi.Var total_workers[2651]>
2652  <gurobi.Var total_workers[2652]>

[476 rows x 1 columns]>
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Model for pipe1:
95676 variables
48552 constraints
47600 general constraints
Input has shape (476, 1)
Output has shape (476, 1)

Pipeline has 2 steps:

--------------------------------------------------------------------------------
Step            Output Shape    Variables              Constraints              
                                 

's#\n\n\n\nfeats = feats.drop(columns=["time"])\nprint(feats)\n\n# pred_constr = add_predictor_constr(m, lin_reg, feats, av)\n# m.Params.NonConvex = 2\n\n\nprint(w.gppd.X)\n\nprint(\n    "Maximum error in approximating the regression {:.6}".format(\n        np.max(pred_constr.get_error())\n    )\n)\n\n\n\n\n\n\n\ndfTerranOpt = dfTerranOpt.loc[:, features]\n\nm.update()\n\ndfTerranOpt[:10]\n\n\n\n\n\n'